# Import packages and main files

In [1]:
import PyPDF2
import os
import re
import requests
import pandas as pd
import numpy as np
from itertools import combinations 
from itertools import permutations
from itertools import chain
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()

In [2]:
courseletlist = (
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F175%2F163757167020201210_Liu_crypto_p2p_lending.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F272%2F1654160257Lesson1-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F273%2F1654160288Lesson1-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F274%2F1654160327Lesson1-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F275%2F1654160374Lesson1-4.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F276%2F1654160475Lesson1-5.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F277%2F1654160518Lesson1-6.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F279%2F1654251498Lesson2-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F278%2F1654160549Lesson1-7.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F280%2F1654251511Lesson2-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F281%2F1654251525Lesson2-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F103%2F20210303+IA+METIS+Reinforcement+Learning.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F107%2F1636712642CATE_meets_ML_Presentation.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163458263420190429+Hae+Ni+LDA+DTM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163646337920210921+Hae+Ni+LDA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F121%2F163646358220210708+Hae+Ni+LDA+extensions.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F108%2F163595835420210530+METIS+WANG+Kalman+Filter.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F110%2F1632126441nodalida2021_summaryQuality_slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F112%2F163664661320211013+Ren+LI+Hae+Expectile+FRM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F114%2F1635233254Shapley.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F118%2F1636625638FRM%40EM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F119%2F163231165920210324+Wan+Hae+Li+k-expectile+clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F130%2F1633104997PAC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F134%2F163368764620210923+Mer+Hae+GAN+Generative+Adversarial+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F139%2F163402565120211012+Kho+Hae+Trespassing+random+forests+with+a+pointed+stick+for+self+defence.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F217%2F1644582711Berlin_short_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F235%2F1649426301Variable+importance+measures+for+RF+.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F179%2F16376558632021122+SBA+JW+Hae+EPF++Quantinar.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F186%2F1645194357Presentation_Quantinar_with_videos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F306%2FBarHan2021_talk.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F181%2F163826827620211130+LI+Hae+Case+based+Bancruptcy+prediction.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F224%2F164728525020220305+LI+Electricity+Market+Coupling.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F196%2F1642599075163458186420200403+METIS+Kho+Hae+Spectral+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F198%2F1642599289163231165920210324+Wan+Hae+Li+k-expectile+clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F141%2F163527012720210526+SAE+NAG+HAE+SIZ+Understanding+jumps+in+high+frequency+digital+asset+markets.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F201%2F16426862241636625638FRM%40EM_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F202%2F1642686343163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F189%2F163958119120211130+Hae+Wan+Kot+ComputerMuseum.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F183%2F1643806658KDE+ill-posed+problems.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F167%2F163699399420211115+Liu+Word+Embeddings+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F157%2F1642778303introduction_data_science.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F180%2F16377653596.+model+assessment+-+part+4+-+appendix.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F128%2F163707553520210331_METIS_Hel_GANs_for_Time_Series.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F127%2F163458156320190528+Cea+Hae+Scagnostics.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F117%2F163458186420200403+METIS+Kho+Hae+Spectral+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F106%2F163458175020200914+Hae+DS2+Data+Science+%26+Digital+Society.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F190%2F1640038524Instruction+for+Creating+Quantlets.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F144%2F1636624210NNCSR_Slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F184%2F163888970320211207+Zin+Reu+Hae+USC+Quantinar+40+min+PDF.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F133%2F163707963820210525_Hae_Xia_Crypto_Indices-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F129%2F163458163120200915+Kim+Hae+Tri+VCRIX.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F123%2F163664570620210922+Mat+Pac+Hae+guide+hedging+CC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F122%2F163283000220210923+Cul+Hae+Pet+Xia+Cryptocurrency+as+an+asset+class.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F116%2F163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F113%2F1632580703FRM+for+Cryptos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F101%2F163170747120210914+Reu+DSF+Digital+Surrogate+Finance+Doc.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F97%2F163458105020210808+METIS+Win+Pricing+Kernel+Risk+Premium.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F163299370720210908_CRC21_Hae_Rodeo_or_Ascot.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F1635155323202109_RoA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F79%2F164604019720210502+Hae+Har+Reu+Understanding+CCs.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F142%2F163627928020211107+Hae+Iva+Mat+Delaunay+Triangulation_A_Shape.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F135%2F1649084960Chapter+1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F138%2F1649094430Chapter+4.pdf',
# problematic:
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F197%2F1642599236164171878820211207+Hae+Zin+Hierarchical+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F241%2F1650632942Biographical+Background+Information.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F210%2F164390414020220130+METIS+Gua+Hae+Model+Selection+Criteria.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F194%2F164171878820211207+Hae+Zin+Hierarchical+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F170%2F163709451820211117+Hae+Qia+Network+Centrality.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F136%2F1649094328Chapter+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F137%2F1633950248Chapter+3.pdf'
)

ids = (["id-{}".format(i+1) for i in range(len(courseletlist))])


stopwords_list = {"i","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd","your","yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers","herself","it","it's","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this",'that',"that'll","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","don't","should","should've","now","d","ll","m","o","re","ve","y","ain","aren","aren't","couldn","couldn't","didn","didn't","doesn","doesn't","hadn","hadn't","hasn","hasn't","haven","haven't","isn","isn't","ma","mightn","mightn't","mustn","mustn't","needn","needn't","shan","shan't","shouldn","shouldn't","wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't"}

id_matrix = pd.DataFrame({'id': ids, 'url': courseletlist})

# Word bank Function definition
## Download function

In [3]:
def download_pdf(file_name, url):

    '''Download a PDF file with an URL'''

    # Define HTTP Headers
    headers = {"User-Agent": "Chrome/51.0.2704.103"}
    
    # Download image
    response = requests.get(url, headers=headers)
    
    # if response is OK download the PDF and store it, else write the status
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)
    
    return

## String creator

In [4]:
def create_string(file_name):
    
    '''Transform a PDF file to a list of string pages'''
    
    # opening the file
    imported_pdf = open(file_name, 'rb')
    
    # convert PDF to readable file
    transformed_pdf = PyPDF2.PdfFileReader(imported_pdf, strict=False)
    
    # get number of pages
    totalpages = transformed_pdf.numPages
    
    # read the data and store in a list
    pdf_output = [transformed_pdf.getPage(i) for i in range(totalpages)]
    
    # extract result
    pdf_output = [pdf_output[i].extractText() for i in range(totalpages)]
    
    return pdf_output, totalpages 

## Cleaning function

In [5]:
def cleaning(file_name):

    '''Initial PDF cleaning procedure'''
    
    pdf_output, totalpages = create_string(file_name)
    
    # # cleaning URLs
    pdf_output = [re.sub(pattern = "http[^ ]*", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning symbols
    pdf_output = [re.sub(pattern = "\\n", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "\W|\d", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "[^a-zA-Z]", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning multispaces
    pdf_output = [re.sub(pattern = "\s{2,}", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning out 1-2-worders
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # lower-casing
    pdf_output = [pdf_output[i].lower() for i in range(totalpages)]
    pdf_output = [[ps.stem(word) for word in sentence.split(" ")] for sentence in pdf_output]
    pdf_output = [' '.join(pdf_output[i]) for i in range(len(pdf_output))]
    
    return pdf_output, totalpages

## Final combined PDF creator

In [6]:
def combined_pdf_creator():
    '''Creating the final master-pdf dataframe'''

    # download the first pdf
    download_pdf(ids[0], courseletlist[0])

    # clean the first pdf
    pdf_output, totalpages = cleaning(ids[0])

    # combine the pdf
    combined_pdf = [' '.join(pdf_output)]

    # iterate on above
    for i in range(1, 70):
        print(i)
        download_pdf(ids[i], courseletlist[i])
        pdf_output, totalpages = cleaning(ids[i])
        combined_pdf.append(' '.join(pdf_output))
        
    return combined_pdf

# Input and search function definitions
## Search input

In [17]:
def input_sequence(input_initial): 

    '''Trimming input search terms to be used for the occurance matrix. The output is a generalized stemmed input form ready for checking and a count of terms for the ngram_range.'''

    # splitting the phrase by pieces
    input_general = input_initial.split(' ')

    # cleaning stopwords
    input_general = [i for i in input_general if i not in stopwords_list]

    # count words
    input_general_count = len(input_general)

    # stem the words
    input_general = [ps.stem(i) for i in input_general]

    # create the additional variations of the phrase
    outer_list = []
    for i in range(0, input_general_count):
        inner_list = [input_general[j : input_general_count-i+j] for j in range(i+1)]
        outer_list.append(inner_list)

    return input_general, input_general_count, outer_list

In [8]:
def general_occurance(input_general_count, combined_pdf): 

    '''Creation of the generalized tfidf occurance matrix based on dynamic parameters.'''

    vectorizer_general = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase=False, stop_words=stopwords_list, ngram_range=(input_general_count, input_general_count))
    X_general = vectorizer_general.fit_transform(combined_pdf)
    xx_general = pd.DataFrame(X_general.toarray(), columns = vectorizer_general.get_feature_names_out())
    return xx_general

In [20]:
def check_for_general(input_initial, input_general_count, outer_list, combined_pdf, number_of_urls):

    '''Main function.'''

    # initiating parameters
    breaker = 0
    xx_general = general_occurance(input_general_count, combined_pdf)
    test_output = xx_general.iloc[:,0]*0
    # first test for full match
    print('search term: ', outer_list[0][0])
    test = ' '.join(outer_list[0][0])
    # if test passed
    if test in list(xx_general.columns):
        # create a ranked index
        ranked_indexes = xx_general[test].sort_values(ascending=False).index
        ranked_indexes = list(ranked_indexes[0:number_of_urls])
        # connect back to urls
        output_url = [courseletlist[i] for i in ranked_indexes]
        print('search result: present\n')
        return output_url
    # if test failed drill-down
    else: 
        print('search result: not present, drill-down\n')
        for y in range(1, len(outer_list)):
            # create a new occurance matrix with new ngrams
            xx_general = general_occurance(input_general_count-y, combined_pdf)
            for u in range(y+1):
                # drill-down phrase test
                print('search term: ', outer_list[y][u])
                test = ' '.join(outer_list[y][u])
                # if test passed
                if test in list(xx_general.columns):
                    # sum the tfidf indexes across multiple matches
                    test_output += xx_general[test]
                    print('search result: present\n')
                    # initiate the exit from the function
                    breaker = 1
                else: 
                    print('search result: not present\n')
            if breaker == 1:
                # order the indexes by highest tfidf
                ranked_indexes = test_output.sort_values(ascending=False).index
                ranked_indexes = list(ranked_indexes[0:number_of_urls])
                # return urls
                output_url = [courseletlist[i] for i in ranked_indexes]
                return output_url

In [ ]:
combined_pdf = combined_pdf_creator()

In [33]:
input_initial = 'Cryptocurrency prices compared To A kittY'
number_of_urls = 5
input_general, input_general_count, outer_list = input_sequence(input_initial)
check_for_general(input_initial, input_general_count, outer_list, combined_pdf, number_of_urls)

search term:  ['cryptocurr', 'price', 'compar', 'to', 'a', 'kitti']
search result: not present, drill-down

search term:  ['cryptocurr', 'price', 'compar', 'to', 'a']
search result: not present

search term:  ['price', 'compar', 'to', 'a', 'kitti']
search result: not present

search term:  ['cryptocurr', 'price', 'compar', 'to']
search result: not present

search term:  ['price', 'compar', 'to', 'a']
search result: not present

search term:  ['compar', 'to', 'a', 'kitti']
search result: not present

search term:  ['cryptocurr', 'price', 'compar']
search result: not present

search term:  ['price', 'compar', 'to']
search result: not present

search term:  ['compar', 'to', 'a']
search result: not present

search term:  ['to', 'a', 'kitti']
search result: not present

search term:  ['cryptocurr', 'price']
search result: not present

search term:  ['price', 'compar']
search result: not present

search term:  ['compar', 'to']
search result: not present

search term:  ['to', 'a']
search resu

['https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F279%2F1654251498Lesson2-1.pdf',
 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F163299370720210908_CRC21_Hae_Rodeo_or_Ascot.pdf',
 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F133%2F163707963820210525_Hae_Xia_Crypto_Indices-2.pdf',
 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F122%2F163283000220210923+Cul+Hae+Pet+Xia+Cryptocurrency+as+an+asset+class.pdf',
 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F129%2F163458163120200915+Kim+Hae+Tri+VCRIX.pdf']